# homuncu_loc image viewer

In [1]:
import napari
from skimage import io
import btrack
import os
from tqdm.auto import tqdm

def find_files_with_basename(target_basename, start_dir='.'):
    """
    Recursively search for files with a given basename in a directory tree.
    
    :param target_basename: The basename to search for.
    :param start_dir: The directory to start the search from. Default is current directory.
    :return: A list of full paths to files matching the given basename.
    """
    matching_files = []
    
    for dirpath, dirnames, filenames in tqdm(os.walk(start_dir), desc = 'Searching start directory for filename'):
        for fname in filenames:
            if os.path.basename(fname) == target_basename:
                full_path = os.path.join(dirpath, fname)
                matching_files.append(full_path)
                
    return matching_files

def find_tif_files(start_dir='.'):
    """
    Recursively search for files with '.tif' extension in a directory tree.
    
    :param start_dir: The directory to start the search from. Default is current directory.
    :return: A list of full paths to files with '.tif' extension.
    """
    matching_files = []
    
    for dirpath, dirnames, filenames in tqdm(os.walk(start_dir), desc='Searching for .tif files'):
        for fname in filenames:
            if fname.endswith('.tif'):
                full_path = os.path.join(dirpath, fname)
                matching_files.append(full_path)
                
    return matching_files

#### Define image fn 

In [7]:
image_fn = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area/run3/23-03-002/20230801_20X_23-03-002A5_DAPI_NKX201_PDPN_ZO-1_Multichannel Z-Stack_20230801_1441.tif'

#### Or search for image basename in directory

In [139]:
image_basename = '20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313.tif'
start_directory = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/'

In [140]:
image_fn_results = find_files_with_basename(image_basename, start_directory)
print(image_fn_results)

Searching start directory for filename: 0it [00:00, ?it/s]

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313.tif']


In [141]:
image_fn = image_fn_results[0]

In [76]:
%%time
image = io.imread(image_fn)
image.shape

CPU times: user 1.21 s, sys: 12.3 s, total: 13.5 s
Wall time: 3min 49s


(61, 2304, 2304, 4)

#### Now do the same for the tracking file

In [142]:
tracks_basename = image_basename.replace('.tif','_z_tracks_masks.h5') #'20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295_z_tracks_masks.h5'
tracks_fn_results = find_files_with_basename(tracks_basename, start_directory)
if len(tracks_fn_results) > 1:
        raise ValueError("Found more than one matching file. Please ensure only one target file exists or refine your search criteria.")
print(tracks_fn_results)

Searching start directory for filename: 0it [00:00, ?it/s]

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313_z_tracks_masks.h5']


In [143]:
tracks_fn = tracks_fn_results[0]

#### Now load the segmentation and z-tracks

In [129]:
with btrack.io.HDF5FileHandler(tracks_fn, 'r', obj_type='obj_type_1') as reader:
    segmentation = reader.segmentation
    tracks = [t for t in reader.tracks if len(t) >= 3]
    objects = reader.objects

[INFO][2023/08/22 01:07:23 pm] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313_z_tracks_masks.h5...
[INFO][2023/08/22 01:07:24 pm] Loading segmentation (4, 2304, 2304)
[INFO][2023/08/22 01:07:24 pm] Loading tracks/obj_type_1
[INFO][2023/08/22 01:07:24 pm] Loading LBEP/obj_type_1
[INFO][2023/08/22 01:07:24 pm] Loading objects/obj_type_1 (12, 5) (12 filtered: None)
[INFO][2023/08/22 01:07:24 pm] Loading objects/obj_type_1 (12, 5) (12 filtered: None)
[INFO][2023/08/22 01:07:24 pm] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313_z_tracks_masks.h5


In [89]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [90]:
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='ID')

In [91]:
viewer = napari.Viewer(title = image_fn)

viewer.add_image(image, channel_axis = -1)
viewer.add_labels(segmentation, visible = False)
viewer.add_labels(recolored_segmentation)
viewer.add_tracks(napari_tracks)

<Tracks layer 'napari_tracks' at 0x7f1841744dc0>

In [92]:
print()

# Retracking

In [95]:
props = ('axis_major_length', # first define some cell properties 
         'axis_minor_length', # this helps improve the tracking
         'eccentricity',      # by comparing similarities between slices/frames
         'area', 
         'orientation',
         'mean_intensity', 
        )
objects = btrack.utils.segmentation_to_objects(
            segmentation = segmentation, # set the masks here 
            intensity_image = image, # provide the image so that the mean intensity can be measured
            properties = props, # provide the cell properties to improve tracker 
            use_weighted_centroid = False, 
            assign_class_ID=False,
            )

[INFO][2023/08/22 11:31:17 am] Localizing objects from segmentation...
100%|█████████████████████████| 61/61 [01:11<00:00,  1.18s/it]
[INFO][2023/08/22 11:32:29 am] Objects are of type: <class 'dict'>
[INFO][2023/08/22 11:32:31 am] ...Found 172246 objects in 61 frames.


In [96]:
with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure('/home/dayn/analysis/models/loc.json')
        ### set max search radius to a very limited radius 
        tracker.max_search_radius = 5
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # use visual features to track
        tracker.features = props
        # append the objects to be tracked
        tracker.append(objects)
        # set the volume
        tracker.volume=((0, segmentation.shape[1]), (0, segmentation.shape[2]), (-1e5, 1e5))
        # track them (in interactive mode)
        tracker.track(step_size=10)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # get the tracks as a python list
        tracks = tracker.tracks

[INFO][2023/08/22 11:34:26 am] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/08/22 11:34:26 am] Starting BayesianTracker session
[INFO][2023/08/22 11:34:26 am] Loading configuration file: /home/dayn/analysis/models/loc.json
[INFO][2023/08/22 11:34:26 am] Objects are of type: <class 'list'>
[INFO][2023/08/22 11:34:30 am] Starting tracking... 
[INFO][2023/08/22 11:34:30 am] Update using: ['VISUAL', 'MOTION']
[INFO][2023/08/22 11:35:26 am] Tracking objects in frames 0 to 9 (of 61)...
[INFO][2023/08/22 12:00:02 pm]  - Timing (Bayesian updates: 43739.25ms, Linking: 94.67ms)
[INFO][2023/08/22 12:00:02 pm]  - Probabilities (Link: 1.00000, Lost: 0.99998)
[INFO][2023/08/22 12:00:02 pm]  - Stats (Active: 5540, Lost: 687, Conflicts resolved: 17401)
[INFO][2023/08/22 12:00:02 pm] Tracking objects in frames 10 to 19 (of 61)...
[INFO][2023/08/22 12:13:11 pm]  - Timing (Bayesian updates: 38414.20ms, Linking: 76.59ms)
[INFO][2023/08/22 12:13:11 pm]  - Probabilities (L

GLPK Integer Optimizer 5.0
87044 rows, 65673 columns, 87824 non-zeros
65673 integer variables, all of which are binary
Preprocessing...
43522 rows, 65673 columns, 87824 non-zeros
65673 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 43522
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
43522 rows, 65673 columns, 87824 non-zeros
*     0: obj =   2.370127093e+05 inf =   0.000e+00 (16782)
*  4325: obj =   1.901104049e+05 inf =   0.000e+00 (12483)
Perturbing LP to avoid stalling [6925]...
*  8993: obj =   1.433704515e+05 inf =   0.000e+00 (7836)
* 13596: obj =   1.196027052e+05 inf =   0.000e+00 (3326)
Removing LP perturbation [17001]...
* 17001: obj =   1.084785872e+05 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+ 17001: mip =     not found yet >=   

[INFO][2023/08/22 12:38:04 pm] Ending BayesianTracker session


In [13]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [14]:
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='ID')

In [16]:
viewer = napari.Viewer(title = fn +'retracked')

# viewer.add_image(image, channel_axis = -1)
viewer.add_labels(segmentation, visible = False)
viewer.add_labels(recolored_segmentation)
viewer.add_tracks(napari_tracks)

<Tracks layer 'napari_tracks' at 0x7f57c78724f0>

# Retrack all 

In [5]:
import napari
from skimage import io
import btrack
import os
from tqdm.auto import tqdm

def find_files_with_basename(target_basename, start_dir='.'):
    """
    Recursively search for files with a given basename in a directory tree.
    
    :param target_basename: The basename to search for.
    :param start_dir: The directory to start the search from. Default is current directory.
    :return: A list of full paths to files matching the given basename.
    """
    matching_files = []
    
    for dirpath, dirnames, filenames in tqdm(os.walk(start_dir), desc = 'Searching start directory for filename'):
        for fname in filenames:
            if os.path.basename(fname) == target_basename:
                full_path = os.path.join(dirpath, fname)
                matching_files.append(full_path)
                
    return matching_files

# from macrohet import notify

def find_tif_files(start_dir='.'):
    """
    Recursively search for files with '.tif' extension in a directory tree.
    
    :param start_dir: The directory to start the search from. Default is current directory.
    :return: A list of full paths to files with '.tif' extension.
    """
    matching_files = []
    
    for dirpath, dirnames, filenames in tqdm(os.walk(start_dir), desc='Searching for .tif files'):
        for fname in filenames:
            if fname.endswith('.tif'):
                full_path = os.path.join(dirpath, fname)
                matching_files.append(full_path)
                
    return matching_files

In [27]:
start_directory = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/'
image_fns = find_tif_files(start_directory)
len(image_fns)

Searching for .tif files: 0it [00:00, ?it/s]

49

In [28]:
image_fns

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_1/DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295.tif',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_1/DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_292.tif',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_1/DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_298.tif',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_1/DAPI-SPC-PDPN-ZO1/Day14_static/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_246.tif',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/h

In [7]:
props = ('axis_major_length',
 'axis_minor_length',
 'eccentricity',
 'area',
 'orientation',
 'mean_intensity')

In [29]:
image_fns = [
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1315.tif',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1320.tif',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1314.tif',]

In [25]:
tracks_fn_results == []

True

In [30]:
for fn in tqdm(image_fns, total = len(image_fns)):
    # image = io.imread(fn)
    image_basename = os.path.basename(fn)
    tracks_basename = image_basename.replace('.tif','_z_tracks_masks.h5') #'20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295_z_tracks_masks.h5'
    tracks_fn_results = find_files_with_basename(tracks_basename, start_directory)
    if len(tracks_fn_results) > 1:
        raise ValueError("Found more than one matching file. Please ensure only one target file exists or refine your search criteria.")
    elif tracks_fn_results == []:
        print(f'No tracking file found for image {image_basename}')
        continue
    tracks_fn = tracks_fn_results[0]
    with btrack.io.HDF5FileHandler(tracks_fn, 'r', obj_type='obj_type_1') as reader:
        objects = reader.objects
        segmentation = reader.segmentation

    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure('/home/dayn/analysis/models/loc.json')
        ### set max search radius to a very limited radius 
        tracker.max_search_radius = 5
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # use visual features to track
        tracker.features = props
        # append the objects to be tracked
        tracker.append(objects)
        # set the volume
        tracker.volume=((0, segmentation.shape[1]), (0, segmentation.shape[2]), (-1e5, 1e5))
        # track them (in interactive mode)
        tracker.track(step_size=10)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # get the tracks as a python list
        tracks = tracker.tracks

    # save out alongside new images
    new_tracks_fn = fn.replace('.tif', '_sc_data.h5').replace('/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/', '/home/dayn/data/homuncu_loc_temp/results/')    
    os.makedirs(os.path.dirname(new_tracks_fn), exist_ok=True)
    # save out 
    with btrack.io.HDF5FileHandler(new_tracks_fn, 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
            writer.write_tracks(tracks)
            writer.write_segmentation(segmentation)
    # notify me
    notify.send_sms(f'{image_basename} complete')

  0%|          | 0/3 [00:00<?, ?it/s]

Searching start directory for filename: 0it [00:00, ?it/s]

[INFO][2023/08/22 05:35:45 PM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1315_z_tracks_masks.h5...
[INFO][2023/08/22 05:35:45 PM] Loading objects/obj_type_1 (13489, 5) (13489 filtered: None)
[INFO][2023/08/22 05:36:05 PM] Loading segmentation (362, 2304, 2304)
[INFO][2023/08/22 05:36:05 PM] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1315_z_tracks_masks.h5
[INFO][2023/08/22 05:36:05 PM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/08/22 05:36:05 PM] Starting BayesianTracker session
[INFO][2023/08/22 05:36:05 PM] Loading configuration file: /home/dayn/analysis/model

GLPK Integer Optimizer 5.0
11156 rows, 8368 columns, 11158 non-zeros
8368 integer variables, all of which are binary
Preprocessing...
5578 rows, 8368 columns, 11158 non-zeros
8368 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 5578
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
5578 rows, 8368 columns, 11158 non-zeros
*     0: obj =   3.649280368e+04 inf =   0.000e+00 (2078)
*  2079: obj =   1.643507169e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  2079: mip =     not found yet >=              -inf        (1; 0)
+  2079: >>>>>   1.643507169e+04 >=   1.643507169e+04   0.0% (1; 0)
+  2079: mip =   1.643507169e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND


[INFO][2023/08/22 05:36:09 PM] Writing objects/obj_type_1
[INFO][2023/08/22 05:36:09 PM] Writing labels/obj_type_1
[INFO][2023/08/22 05:36:09 PM] Loading objects/obj_type_1 (11722, 5) (11722 filtered: None)
[INFO][2023/08/22 05:36:09 PM] Writing properties/obj_type_1/axis_major_length (11722,)
[INFO][2023/08/22 05:36:09 PM] Writing properties/obj_type_1/axis_minor_length (11722,)
[INFO][2023/08/22 05:36:09 PM] Writing properties/obj_type_1/eccentricity (11722,)
[INFO][2023/08/22 05:36:09 PM] Writing properties/obj_type_1/area (11722,)
[INFO][2023/08/22 05:36:09 PM] Writing properties/obj_type_1/orientation (11722,)
[INFO][2023/08/22 05:36:09 PM] Writing properties/obj_type_1/mean_intensity (11722, 4)
[INFO][2023/08/22 05:36:09 PM] Writing properties/obj_type_1/mtb_area (11722,)
[INFO][2023/08/22 05:36:09 PM] Writing properties/obj_type_1/mtb_status (11722,)
[INFO][2023/08/22 05:36:09 PM] Writing tracks/obj_type_1
[INFO][2023/08/22 05:36:09 PM] Writing dummies/obj_type_1
[INFO][2023/08/

...

[INFO][2023/08/22 05:37:00 PM] Opening HDF file: /home/dayn/data/homuncu_loc_temp/results/image analysis_Nathan/Job_Mtb area/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1320_sc_data.h5...
[INFO][2023/08/22 05:37:00 PM] Writing objects/obj_type_1
[INFO][2023/08/22 05:37:00 PM] Writing labels/obj_type_1
[INFO][2023/08/22 05:37:00 PM] Loading objects/obj_type_1 (10870, 5) (10870 filtered: None)
[INFO][2023/08/22 05:37:00 PM] Writing properties/obj_type_1/axis_major_length (10870,)
[INFO][2023/08/22 05:37:00 PM] Writing properties/obj_type_1/axis_minor_length (10870,)
[INFO][2023/08/22 05:37:00 PM] Writing properties/obj_type_1/eccentricity (10870,)
[INFO][2023/08/22 05:37:00 PM] Writing properties/obj_type_1/area (10870,)
[INFO][2023/08/22 05:37:00 PM] Writing properties/obj_type_1/orientation (10870,)
[INFO][2023/08/22 05:37:00 PM] Writing properties/obj_type_1/mean_intensity (10870, 4)
[INFO][2023/08/22 05:37:00 PM] Writing properties/obj_type_1

Searching start directory for filename: 0it [00:00, ?it/s]

[INFO][2023/08/22 05:37:31 PM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1314_z_tracks_masks.h5...
[INFO][2023/08/22 05:37:31 PM] Loading objects/obj_type_1 (12239, 5) (12239 filtered: None)
[INFO][2023/08/22 05:37:44 PM] Loading segmentation (303, 2304, 2304)
[INFO][2023/08/22 05:37:44 PM] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1314_z_tracks_masks.h5
[INFO][2023/08/22 05:37:44 PM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/08/22 05:37:44 PM] Starting BayesianTracker session
[INFO][2023/08/22 05:37:44 PM] Loading configuration file: /home/dayn/analysis/model

GLPK Integer Optimizer 5.0
10944 rows, 8208 columns, 10944 non-zeros
8208 integer variables, all of which are binary
Preprocessing...
5472 rows, 8208 columns, 10944 non-zeros
8208 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 5472
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
5472 rows, 8208 columns, 10944 non-zeros
*     0: obj =   3.579909169e+04 inf =   0.000e+00 (2115)
*  2115: obj =   1.509093650e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  2115: mip =     not found yet >=              -inf        (1; 0)
+  2115: >>>>>   1.509093650e+04 >=   1.509093650e+04   0.0% (1; 0)
+  2115: mip =   1.509093650e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND


[INFO][2023/08/22 05:37:47 PM] Writing objects/obj_type_1
[INFO][2023/08/22 05:37:47 PM] Writing labels/obj_type_1
[INFO][2023/08/22 05:37:47 PM] Loading objects/obj_type_1 (10736, 5) (10736 filtered: None)
[INFO][2023/08/22 05:37:47 PM] Writing properties/obj_type_1/axis_major_length (10736,)
[INFO][2023/08/22 05:37:47 PM] Writing properties/obj_type_1/axis_minor_length (10736,)
[INFO][2023/08/22 05:37:47 PM] Writing properties/obj_type_1/class_id (10736,)
[INFO][2023/08/22 05:37:47 PM] Writing properties/obj_type_1/eccentricity (10736,)
[INFO][2023/08/22 05:37:47 PM] Writing properties/obj_type_1/area (10736,)
[INFO][2023/08/22 05:37:47 PM] Writing properties/obj_type_1/orientation (10736,)
[INFO][2023/08/22 05:37:47 PM] Writing properties/obj_type_1/mean_intensity (10736, 4)
[INFO][2023/08/22 05:37:47 PM] Writing properties/obj_type_1/mtb_area (10736,)
[INFO][2023/08/22 05:37:47 PM] Writing properties/obj_type_1/mtb_status (10736,)
[INFO][2023/08/22 05:37:47 PM] Writing tracks/obj_t